In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Configurações do Pandas
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.precision', 2)

In [17]:
df = dados.copy(deep=True)
df = df.loc[df['procedencia'] == '0143096-PORTO ALEGRE (RS)']

df = df.groupby(['tipo','mes'])[['volume','total']].agg({'volume':'sum','total':'sum'}).reset_index()
df = df.pivot_table(index='mes', columns='tipo', values=['volume','total']).fillna(0)

df['Total Volume'] = df[('volume','PRODUTOR')] + df[('volume','ATACADISTA')]
df['Total Valor'] = df[('total','PRODUTOR')] + df[('total','ATACADISTA')]

# Adiciona a linha de soma anual corretamente
df.loc['Soma Ano 2024'] = df.sum(numeric_only=True)

df.to_excel(r'~/Desktop/Dados de Comercialização Porto Alegre 2024.xlsx')


In [4]:
%%sql
SELECT ibge, cidade as procedencia FROM cad_cidades;

,ibge,procedencia
0,1100023,0111001-ARIQUEMES (RO)
1,1100049,0111002-CACOAL (RO)
2,1100056,0111003-CEREJEIRAS (RO)
3,1100064,0111004-COLORADO DO OESTE (RO)
4,1100080,0111005-COSTA MARQUES (RO)
...,...,...
4519,14,2900000-(PORTUGAL)
4520,7,3100000-(NOVA ZELANDIA)
4521,9898989,3500000-(GRECIA)
4522,9898989,3600000-(COLOMBIA)


In [7]:
dados = pd.read_parquet('D:\Codigos\CEASA-Dados\Bases de dados\Fechamento 2024.parquet')

In [8]:
dados = dados.groupby(['estado','procedencia'])[['volume','total']].sum().reset_index()
dados

,estado,procedencia,volume,total
0,ALAGOAS,0127005-ARAPIRACA (AL),"262,678.00","1,365,890.00"
1,ALAGOAS,0127022-COITE DE NOIA (AL),"44,200.00","238,000.00"
2,ALAGOAS,0127029-FEIRA GRANDE (AL),"1,175,775.00","4,336,823.30"
3,ALAGOAS,0127043-JUNQUEIRO (AL),"1,202,760.00","6,408,961.11"
4,ALAGOAS,0127093-TAQUARANA (AL),"13,750.00","66,000.00"
...,...,...,...,...
703,TOCANTINS,0155001-MIRACEMA DO TOCANTINS (TO),"14,000.00","32,200.00"
704,TOCANTINS,0155008-LAGOA DA CONFUSAO (TO),"57,500.00","139,850.00"
705,TOCANTINS,0155018-CRISTALANDIA (TO),"59,500.00","135,250.00"
706,TOCANTINS,0155022-ALVORADA (TO),"83,189.00","175,015.80"


In [9]:
df = pd.merge(left=dados, right=ibge, how='left', on='procedencia')
df

,estado,procedencia,volume,total,ibge
0,ALAGOAS,0127005-ARAPIRACA (AL),"262,678.00","1,365,890.00",2700300
1,ALAGOAS,0127022-COITE DE NOIA (AL),"44,200.00","238,000.00",2702009
2,ALAGOAS,0127029-FEIRA GRANDE (AL),"1,175,775.00","4,336,823.30",2702603
3,ALAGOAS,0127043-JUNQUEIRO (AL),"1,202,760.00","6,408,961.11",2704005
4,ALAGOAS,0127093-TAQUARANA (AL),"13,750.00","66,000.00",2709103
...,...,...,...,...,...
703,TOCANTINS,0155001-MIRACEMA DO TOCANTINS (TO),"14,000.00","32,200.00",1713205
704,TOCANTINS,0155008-LAGOA DA CONFUSAO (TO),"57,500.00","139,850.00",1711902
705,TOCANTINS,0155018-CRISTALANDIA (TO),"59,500.00","135,250.00",1706100
706,TOCANTINS,0155022-ALVORADA (TO),"83,189.00","175,015.80",1700707


In [10]:
df = df.rename(columns={'total':'valor','procedencia':'cidade'})
df

,estado,cidade,volume,valor,ibge
0,ALAGOAS,0127005-ARAPIRACA (AL),"262,678.00","1,365,890.00",2700300
1,ALAGOAS,0127022-COITE DE NOIA (AL),"44,200.00","238,000.00",2702009
2,ALAGOAS,0127029-FEIRA GRANDE (AL),"1,175,775.00","4,336,823.30",2702603
3,ALAGOAS,0127043-JUNQUEIRO (AL),"1,202,760.00","6,408,961.11",2704005
4,ALAGOAS,0127093-TAQUARANA (AL),"13,750.00","66,000.00",2709103
...,...,...,...,...,...
703,TOCANTINS,0155001-MIRACEMA DO TOCANTINS (TO),"14,000.00","32,200.00",1713205
704,TOCANTINS,0155008-LAGOA DA CONFUSAO (TO),"57,500.00","139,850.00",1711902
705,TOCANTINS,0155018-CRISTALANDIA (TO),"59,500.00","135,250.00",1706100
706,TOCANTINS,0155022-ALVORADA (TO),"83,189.00","175,015.80",1700707


In [11]:
df.to_csv(r'~/Desktop/volume_comercializado_2024.csv')